In [2]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar
import json

In [3]:
df = pd.read_csv('../debit_transactions_processed.csv')

In [4]:
def calculate_monthly_pace(df: pd.DataFrame, current_date_str: str) -> dict:
    """
    Calculates the "Monthly Pace" metric for the current month up to a specific date
    and determines the trend in relation to the total spending of the previous month.

    Args:
        df (pd.DataFrame): DataFrame containing transactions with 'date' (datetime)
                           and 'amount' (float) columns.
        current_date_str (str): The reference date in 'YYYY-MM-DD' format to
                                calculate the "pace" up to that day.

    Returns:
        dict: A dictionary (JSON format) with the metric name, the calculated value,
              and the trend ('upward', 'downward', 'stable', 'insufficient_data').
    """
    try:
        df_debits = df.copy()

        # Set Date
        current_date = pd.to_datetime(current_date_str)
        # Guarantees that 'date' column is in datetime format
        df['date'] = pd.to_datetime(df['date'])

        # Check if current_date is within the dataset range
        max_date_in_dataset = df['date'].max()
        if current_date > max_date_in_dataset:
            return {
                "metric_name": "Monthly Pace",
                "value": 0.0,
                "trend": "error",
                "error_message": f"Requested date ({current_date_str}) is beyond available data (up to {max_date_in_dataset.strftime('%Y-%m-%d')})"
            }

        # Define start and end dates for current and previous months
        start_of_current_month = current_date.replace(day=1)
        start_of_previous_month = start_of_current_month - relativedelta(months=1)
        end_of_previous_month = start_of_current_month - relativedelta(days=1)

        # Monthly Pace Calculation
        # Filter to a specific date range and only debit transactions
        current_month_transactions_to_date = df_debits[
            (df_debits['date'] >= start_of_current_month) &
            (df_debits['date'] <= current_date)
        ]

        total_spent_to_date = float(current_month_transactions_to_date['amount_abs'].sum())
        current_day_of_month = current_date.day
        _, total_days_in_current_month = calendar.monthrange(current_date.year, current_date.month)

        monthly_pace_current = 0.0
        if current_day_of_month > 0:
            avg_daily_spending = total_spent_to_date / current_day_of_month
            monthly_pace_current = avg_daily_spending * total_days_in_current_month

        # Total Spent Previous Month
        previous_month_transactions = df_debits[
            (df_debits['date'] >= start_of_previous_month) &
            (df_debits['date'] <= end_of_previous_month)
        ]
        total_spent_previous_month = float(previous_month_transactions['amount_abs'].sum())

        # Determine Trend
        trend = "stable"
        # Calculate trend only if there is data for the previous month
        if total_spent_previous_month > 0:
            tolerance = 0.05 * total_spent_previous_month
            if monthly_pace_current > total_spent_previous_month + tolerance:
                trend = "upward"
            elif monthly_pace_current < total_spent_previous_month - tolerance:
                trend = "downward"
        else:
            # No data for previous month - insufficient historical data
            trend = "insufficient_data"

        # Output Result
        result = {
            "metric_name": "Monthly Pace",
            "value": round(monthly_pace_current, 2),
            "trend": trend
        }

        return result

    except Exception as e:
        print(f"Error calculating the metric: {e}")
        # Return error structure
        return {
            "metric_name": "Monthly Pace",
            "value": 0.0,
            "trend": "error",
            "error_message": str(e)
        }

In [9]:
calculate_monthly_pace(df, '2025-08-05')

{'metric_name': 'Monthly Pace', 'value': 75480.29, 'trend': 'upward'}